<a href="https://colab.research.google.com/github/Tavo826/DataScience/blob/main/AgeAndGenderDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El objetivo es detectar el género y la edad de una persona a través de una imagen usando una CNN.

Los géneros a predecir son 'male' y 'female' y la edad a predecir estará entre estos rangos (0 - 2), (4 - 6), (8 - 12), (15 - 20), (25 - 32), (38 - 43), (48 - 53), (60 - 100), lo que significa que la capa final será una softmax de 8 neuronas.

La arquitectura de la red será:
 * Capa convolucional: 96 neuronas, kernel size 7
 * Capa convolucional: 256 neuronas, kernel size 6
 * Capa convolucional: 384 neuronas, kernel size 3

Seguidas de 2 capas densas, cada una con 512 neuronas y una capa final softmax

## Dataset

El conjunto de datos datos contiene fotos de rostros e incluye varias condiciones del mundo real como ruido, iluminación, pose, y apariencia. Son un total de 26580 fotos de 2284 personnas en 8 rangos de edad

Para la detección de rostros se cuenta con un archivo .pb (formato binario) que mantiene la definición de la imagen y los pesos entrenados del modelo. El archivo .pbtxt contiene el protocol buffer en formato de texto. El .prototxt descriibe la configuración o estructura de la red y el archivo .caffemodel define los estados internos de los parámetros de las capas o sea los pesos de la red.

In [ ]:
import os

import tensorflow as tf

os.chdir('/content/drive/MyDrive/Colab Notebooks/Data Science/Detección de edad y género')

## Face Detection

### .pb

In [ ]:
with tf.compat.v1.Session() as sess:
  with tf.io.gfile.GFile('saved_model.pb', 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
  sess.graph.as_default()
  with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name='')

  graph_nodes = [n for n in graph_def.node]
  names = []
  for t in graph_nodes:
    names.append(t.name)  
  print(len(names))
  print('Nombres de las capas de la red:')
  print(names)

NameError: ignored

In [ ]:
with tf.io.gfile.GFile('saved_model.pb', 'rb') as f:
  graph_def = tf.compat.v1.GraphDef()
  graph_def.ParseFromString(f.read())
  tf.import_graph_def(graph_def, name='')

constant_values = {}

with tf.compat.v1.Session() as sess:
  #all_vars = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES)
  constant_ops = [op for op in sess.graph.get_operations() if op.type == 'Const']
  for constant_op in constant_ops:
    constant_values[constant_op.name] = sess.run(constant_op.outputs[0])

NameError: ignored

In [ ]:
#Pesos y bias de la red
print(len(constant_values))
#print(constant_values)

### .pbtxt

In [ ]:
from google.protobuf import text_format

with tf.io.gfile.GFile('opencv_face_detector.pbtxt', 'r') as f:
  graph_def = tf.compat.v1.GraphDef()

  file_content = f.read()

  text_format.Merge(file_content, graph_def)

graph_def

### .prototxt


In [ ]:
#age_deploy
age_deploy = open('age_deploy.prototxt')
age = age_deploy.readlines()

for line in age:
  print(line)

In [ ]:
#gender_deploy
gender_deploy = open('gender_deploy.prototxt')
gender = gender_deploy.readlines()

for line in gender:
  print(line)

### .caffemodel

In [ ]:
!apt install caffe-cpu

In [ ]:
import caffe
net = caffe.Net('/content/drive/MyDrive/Colab Notebooks/Data Science/Detección de edad y género/age_deploy.prototxt', 
                '/content/drive/MyDrive/Colab Notebooks/Data Science/Detección de edad y género/age_net.caffemodel',
                caffe.TEST)
pynet_ = []

#Recorriendo las capas de la red
for li in range(len(net.layers)):
  layer = {}
  layer['name'] = net._layer_names[li]
  #Nombre y forma del la entrada de la capa
  layer['bottoms'] = [(net._blob_names[bi], net.blobs[net._blob_names[bi]].data.shape) for bi in list(net._bottom_ids(li))]
  #Nombre y forma de la salida de la capa
  layer['tops'] = [(net._blob_names[bi], net.blobs[net._blob_names[bi]].data.shape) for bi in list(net._top_ids(li))]
  #Tipo de capa
  layer['type'] = net.layers[li].type
  #Parámetros internos de la capa
  layer['weights'] = [net.layers[li].blobs[bi].data[...] for bi in range(len(net.layers[li].blobs))]
  pynet_.append(layer)

pynet_

### Steps

Se utiliza la librería argparse para crear un generador de argumentos para así obtener el argumento de la imagen desde el símbolo del sistema.

Para rostro, edad y género, se inicializa el búfer del protocolo y modelo

Se inicializa los valores medios para el modelo y la lista con el rango de edades y géneros para clasificar

Se usa el método readNet() para cargar las redes. El primer parámetro contiene pesos entrenados y el segundo lleva la configuración de la red.

Se captura la cámara de video en caso de que se quiera clasificar lo que se muestre en la webcam

Se almacena el contenido dentro hasFrame y frame

Se hace una llamada a la función highlightFace() con los parámetros de faceNet y frame, lo que esto retorne será guardado en resultImg y faceBoxes

El modelo net (faceNet) es la red neuronal profunda de reconocimiento facial

* Se crea una copia superficial del marco que bordea el rostro y se obtiene el alto y ancho
* Se crea una *gota* de la copia superficial
* Configura la entrada y la ingresa a la red
* faceBoxes es una lista vacía. Por cada valor entre 0 a 127, se define la confianza (0-1). Donde se encuentre una confianza mayor a la establecida (0.7), se obtienen las coordenadas y se agregan faceBoxes en forma de lista
* Finalmente se ubica un rectángulo en la imagen para cada una de las listas de coordenadas y retorna la copia superficial y la lista faceBoxes

Se define el rostro y se crea un objeto de 4 dimensiones desde la imagen, al hacer esto, se escala, se redimensiona y se pasan los valores medios

Se alimenta la entrada para que la red obtenga la confianza de las clases. La más alta, es la seleccionada

In [ ]:
import cv2
import math
import argparse

def highlightFace(netm frame, conf_threshold=0.7):
  frameOpencvDnn = frame.copy
  frameHeight = frameOpencvDnn.shape[0]
  frameWidth = frameOpencvDnn.shape[1]
  blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300,300), [104,117,123], True, False)

  net.setInput(blob)
  detections = net.forward()
  faceBoxes = []
  for i in range(detections.shape[2]):
    confidence = detections[0,0,i,2]
    if confidence > conf_threshold:
      x1 = int(detections[0,0,i,3] * frameWidth)
      y1 = int(detections[0,0,i,4] * frameHeight)
      x2 = int(detections[0,0,i,5] * frameWidth)
      y2 = int(detections[0,0,i,6] * frameHeight)
      faceBoxes.append([x1,y1,x2,y2])
      cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
  
  return frameOpencvDnn, faceBoxes

parser = argparse.ArgumentParser()
parser.add_argument('--image')

args = parser.parse_args()

faceProto = 'open_face_detector.pbtxt'
faceModel = 'saved_model.pb'
ageProto = 'age_deploy.prototxt'
ageModel = 'age_net.caffemodel'
genderProto = 'gender_deploy.prototxt'
genderModel = 'gender_net.caffemodel'

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

faceNet = cv2.dnn.readNet(faceModel, faceProto)
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

video = cv2.Videocapture(args.image if args.image else 0)
padding = 20
while cv2.waitKey(1) < 0:
  hasFrame, frame = video.read()
  if not hasFrame:
    cv2.waitKey()
    break

  resultImg, faceBoxes = highlightFace(faceNet, frame)
  if nos faceBoxes:
    print('No face detected')

  for faceBox in faceBoxes:
    face = frame[max(0, faceBox[1] - padding):
                min(faceBox[3] + padding, frame.shape[0] - 1, max(0,faceBox[0] - padding))
                :min(faceBox[2] + padding, frame.shape[1] - 1)]
            
    blob = cv2.dnn.blobFromImage(face, 1.0, (277,277), MODEL_MEAN_VALUES, swapRB=False)
    genderNet.setInput(blob)
    genderPreds = genderNet.forward()
    gender = genderList[genderPreds[0].argmax()]
    print(f'Gender: {gender}')

    ageNet.setInput(blob)
    agePreds = ageNet.forward()
    age = ageList[agePreds[0].argmax()]
    print(f'Age: {age[1:-1]} years')

    cv2.putText(resultImg, f'{gender}, {age}', (faceBox[0], faceBo[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2, cv2.LINE_AA)
    cv2-imshow("Detecting age and gender", resultImg)